# Imports 

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

from ml_bets.constants import FEATURES_PATH
from ml_bets.features.features import Features
from ml_bets.modeling.match_model import PipelineDatasets, run_pycaret_setup
from ml_bets.supplementary.functions import create_dataset, add_targets, add_columns_table, train_test_split
from feature_selection.feature_selection import FeatureSelection

from pycaret.classification import (add_metric,
    create_model,
    finalize_model,
    optimize_threshold,
    save_model,
    compare_models, 
    evaluate_model,
    get_config,
    setup,
    tune_model,
)

# Parameters 

In [ ]:
metric_param = {
        "Accuracy": 0.1,
        "AUC": 0.1,
        "Recall": 0.1,
        "Precision": 0.1,
        "F1": 0.1,
        "Kappa": -1.0,
        "MCC": -1.0,
    }  # NEED TO BE FIXED. REWRITE PARAMETERS!!!!! 
metrics_list = ["Accuracy", "AUC", "Recall", "Precision", "F1", "Kappa", "MCC"]
user_date = "20-October-2021"
user_target = 'goals_2.5'
cutoff = user_target.split("_")[-1]
ignore_features = ['month','date','competition','season_x','month_start_or_end','raw_match_id','season_y']
fold_strategy = "timeseries"

### Pycaret setup kwargs 

In [ ]:
setup_kwargs = dict(
        preprocess=True,
        train_size=0.75,
        session_id=123,
        normalize=True,
        transformation=True,
        ignore_low_variance=True,
        remove_multicollinearity=True,
        multicollinearity_threshold=0.75,
        n_jobs=-1,
        use_gpu=False,
        profile=False,
        ignore_features=ignore_features,
        fold_strategy=fold_strategy,
        remove_perfect_collinearity=True,
        create_clusters=False,
        fold=3,
        feature_selection=True,
        # you can use this to keep the 95 % most relevant features (fat_sel_threshold)
        feature_selection_threshold=0.5,
        combine_rare_levels=False,
        rare_level_threshold=0.02,
        pca=False,
        pca_method="kernel",
        pca_components=30,
        polynomial_features=False,
        polynomial_degree=2,
        polynomial_threshold=0.01,
        trigonometry_features=False,
        remove_outliers=True,
        outliers_threshold=0.01,
        feature_ratio=False,
        feature_interaction=False,
        # Makes everything slow AF. use to find out possibly interesting features
        interaction_threshold=0.01,
        fix_imbalance=True,
        log_experiment=False,
        verbose=False,
        silent=True,
        experiment_name="lagstest",
    )

# Creation dataset (standard)

In [ ]:
df = create_dataset()
df = add_targets(df=df, cutoff=cutoff)
# Train-test split
train, test = train_test_split(df=df, sep=user_date) # insert table to evaluate and separation date 

## Pycaret evaluation 

In [ ]:
numerics = ["int16", "int32", "int64", "float16", "float32", "float64", "int", "float"]
num_cols = train.select_dtypes(include=numerics).columns.tolist()
setup_kwargs['numeric_features'] = num_cols 

In [ ]:
_ = setup(data=train, target=user_target, **setup_kwargs)
x_train = get_config('X_train')

In [ ]:
top_models = compare_models(
            n_select=5,
            sort='AUC',
            exclude=["qda", "knn", "nb", 'catboost'],
            verbose=True,
        )

In [ ]:
evaluate_model(top_models[0])

## Finalize model

In [ ]:
final = finalize_model(top_models[0]) # Choose the preferred model. Best performance model selected by default. 

## Metrics against test

In [ ]:
predictions = predict_model(final, data=test)

In [ ]:
metrics_dict = {
    metric: check_metric(
        actual=predictions[user_target], 
        prediction=predictions['Label'], 
        metric=metric
    ) for metric in metrics_list
}

In [ ]:
metrics_df = pd.DataFrame(columns=metrics_dict.keys(), index=pd.Index(range(1)))
metrics_df.index = ['standard_model']
for col, val in metrics_dict.items():
    metrics_df.loc['standard_model', col] = val 
metrics_df

# Creation dataset (with new columns)

### Preparation 

In [ ]:
features = Features(output=features_path)
examples = features.create()
examples = examples[~examples[f"prob_under_goals_{cutoff}"].isna()]
# Initialize PipelineDataset class
pipe_ds = PipelineDatasets(
    features=features,
    target=user_target,
    examples=examples,
    test_size=user_date,
)
n_train, n_test = pipe_ds.train_data, pipe_ds.test_data 

## Referee table. Merging dataframe 

In [ ]:
current_path = os.getcwd() 
path = Path(current_path).parent / 'data/features/referee_table.csv'
referee_table = pd.read_csv(path)

In [ ]:
referee_table = add_columns_table(feat=features, df=referee_table) # add unique id_match to referee table 

In [ ]:
referee_cols  = referee_table.columns.tolist() 
referee_cols.remove('competition')
referee_cols.remove('date') # Remove repeated columns 
referee_train = pd.merge(n_train, referee_table[referee_cols], left_index=True, right_index=True, how='inner') # Referee data + train old data
referee_test = pd.merge(n_test, referee_table[referee_cols], left_index=True, right_index=True, how='inner') # Referee data + test old data

## Selection of features

In [ ]:
feat_sel = FeatureSelection(target=user_target, user_df=referee_train, target_features=0.4, filter_metrics=metric_param)
selected_features = feat_sel.repeat_pipeline()

## Pycaret evaluation

In [ ]:
referee_train = referee_train[selected_features + [user_target]]
numerics = ["int16", "int32", "int64", "float16", "float32", "float64", "int", "float"]
num_cols = referee_train.select_dtypes(include=numerics).columns.tolist()
setup_kwargs['numeric_features'] = num_cols 

In [ ]:
_ = setup(data=referee_train, target=user_target, **setup_kwargs)
x_train = get_config('X_train')

In [ ]:
top_models = compare_models(
            n_select=5,
            sort='AUC',
            exclude=["qda", "knn", "nb", 'catboost'],
            verbose=True,
        )

In [ ]:
evaluate_model(top_models[0])

## Finalize model

In [ ]:
final = finalize_model(top_models[0]) # Choose the preferred model. Best performance model selected by default. 

## Metrics against test

In [ ]:
predictions = predict_model(final, data=referee_test[selected_features + [user_target]])

In [ ]:
metrics_dict = {
    metric: check_metric(
        actual=predictions[user_target], 
        prediction=predictions['Label'], 
        metric=metric
    ) for metric in metrics_list
}

In [ ]:
new_model_metrics_df = pd.DataFrame(columns=metrics_dict.keys(), index=pd.Index(range(1)))
new_model_metrics_df.index = ['new_feat_model']
for col, val in metrics_dict.items():
    new_model_metrics_df.loc['new_feat_model', col] = val 
new_model_metrics_df

In [ ]:
pd.concat([metrics_df, new_model_metrics_df])